In [15]:
!pip install selenium webdriver-manager copernicusmarine fitz frontend pdf2image

In [16]:
!apt-get update --fix-missing
!apt-get install -y poppler-utils

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://packages.cloud.google.com/apt gcsfuse-jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 257 kB in 2s (139 kB/s)
Reading package lists... Done
W: https://packages.cloud.google.com/apt/dists/gcsfuse-jammy/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), 

In [17]:
# prompt: add code to create all the tables if they do not exist with proper column names and types as existing.

import pandas_gbq
from google.cloud import bigquery

project_id = "ocean-data-e68c2"
dataset_id = "ocean_data"

client = bigquery.Client(project=project_id)
dataset_ref = client.dataset(dataset_id)

tables_schema = {
    "link": [
        {"name": "id", "type": "FLOAT64"},
        {"name": "english", "type": "STRING"},
        {"name": "tamil", "type": "STRING"},
        {"name": "image", "type": "STRING"},
    ],
    "current_data_2": [
        {"name": "depth", "type": "FLOAT64"},
        {"name": "latitude", "type": "FLOAT64"},
        {"name": "longitude", "type": "FLOAT64"},
        {"name": "time", "type": "STRING"},
        {"name": "thetao", "type": "FLOAT64"},
    ],
    "pfz": [
        {"name": "WKT", "type": "STRING"},
        {"name": "name", "type": "STRING"},
        {"name": "description", "type": "STRING"},
    ],
    "wind_data": [
        {"name": "time", "type": "STRING"},
        {"name": "latitude", "type": "FLOAT64"},
        {"name": "longitude", "type": "FLOAT64"},
        {"name": "eastward_wind", "type": "FLOAT64"},
        {"name": "northward_wind", "type": "FLOAT64"},
    ],
    "current_data": [
        {"name": "depth", "type": "FLOAT64"},
        {"name": "latitude", "type": "FLOAT64"},
        {"name": "longitude", "type": "FLOAT64"},
        {"name": "time", "type": "STRING"},
        {"name": "uo", "type": "FLOAT64"},
        {"name": "vo", "type": "FLOAT64"},
        {"name": "fetch_date", "type": "DATE"},
    ],
    "biogeochemical_data": [
        {"name": "depth", "type": "FLOAT64"},
        {"name": "latitude", "type": "FLOAT64"},
        {"name": "longitude", "type": "FLOAT64"},
        {"name": "time", "type": "STRING"},
        {"name": "nppv", "type": "FLOAT64"},
        {"name": "o2", "type": "FLOAT64"},
    ],
}


for table_name, schema in tables_schema.items():
    table_ref = dataset_ref.table(table_name)
    try:
        client.get_table(table_ref)
        print(f"Table {table_name} already exists.")
    except:
        table = bigquery.Table(table_ref, schema=schema)
        client.create_table(table)
        print(f"Table {table_name} created.")

Table link already exists.
Table current_data_2 already exists.
Table pfz already exists.
Table wind_data already exists.
Table current_data already exists.
Table biogeochemical_data already exists.


In [18]:
# prompt: Truncate all tables in the ocean_data
from google.cloud import bigquery
import pandas_gbq

# Initialize BigQuery client
client = bigquery.Client()
dataset_id = "ocean-data-e68c2.ocean_data"

# List of tables to truncate
tables = [
    "current_data",
    "biogeochemical_data",
    "wind_data",
    "current_data_2",
    "current_data_3",
    "pfz",
    "link"
]

for table in tables:
    try:
        # Construct the SQL query for truncation
        sql = f"TRUNCATE TABLE `{dataset_id}.{table}`"

        # Execute the query using pandas_gbq
        pandas_gbq.read_gbq(sql, project_id=client.project, dialect="standard")
        print(f"Table {table} truncated successfully.")

    except Exception as e:
        print(f"Error truncating table {table}: {e}")

Downloading: |          |
Table current_data truncated successfully.
Downloading: |          |
Table biogeochemical_data truncated successfully.
Downloading: |          |
Table wind_data truncated successfully.
Downloading: |          |
Table current_data_2 truncated successfully.
Error truncating table current_data_3: Reason: 404 POST https://bigquery.googleapis.com/bigquery/v2/projects/ocean-data-e68c2/queries?prettyPrint=false: Not found: Table ocean-data-e68c2:ocean_data.current_data_3 was not found in location US
Downloading: |          |
Table pfz truncated successfully.
Downloading: |          |
Table link truncated successfully.


In [19]:
from google.cloud import bigquery
import copernicusmarine
import pandas as pd
from datetime import datetime
from datetime import datetime,timedelta
a=datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
# Initialize BigQuery client
client = bigquery.Client()
dataset_id = "ocean-data-e68c2.ocean_data"
a_wind = (datetime.now() - timedelta(days=2)).strftime("%Y-%m-%dT%H:%M:%S")
# Function to fetch and display data
def fetch_and_display_data(query_params):
    try:
        df = copernicusmarine.read_dataframe(**query_params).reset_index().dropna()
        if not df.empty:
            print("Data fetched successfully:")
            print(df)  # Display the fetched DataFrame
            return df  # Return the dataframe if needed for further processing.
        else:
            print("No data fetched for the specified parameters.")
            return None
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

# Function to push data to BigQuery
def push_data_to_bigquery(df, dataset_id, table_name):
    if df is not None:
        try:
            if "time" in df.columns:
                df["time"] = df["time"].astype(str)
            client.load_table_from_dataframe(df, f"{dataset_id}.{table_name}").result()
            print(f"Pushed data to {dataset_id}.{table_name}")
        except Exception as e:
            print(f"Error pushing data to BigQuery: {e}")

# Query parameters for each dataset
current_data_params = {
    "dataset_id": "cmems_mod_glo_phy-cur_anfc_0.083deg_P1D-m",
    "minimum_longitude": 71.67651854792956,
    "maximum_longitude": 83.99011977588334,
    "minimum_latitude": 5.554121417215164,
    "maximum_latitude": 17.074981057486614,
    "variables": ["uo", "vo"],
    "start_datetime": a,
    "end_datetime": a,
    "minimum_depth": 0.49402499198913574,
    "maximum_depth": 0.49402499198913574,
    "username": "asas",
    "password": "dssd"
}

biogeochemical_data_params = {
    "dataset_id": "cmems_mod_glo_bgc-bio_anfc_0.25deg_P1D-m",
    "variables": ["nppv", "o2"],
    "minimum_longitude": 71.67651854792956,
    "maximum_longitude": 83.99011977588334,
    "minimum_latitude": 5.554121417215164,
    "maximum_latitude": 17.074981057486614,
    "start_datetime":a,
    "end_datetime": a,
    "minimum_depth": 0.4940253794193268,
    "maximum_depth": 0.4940253794193268,
    "username": "asas",
    "password": "dssd"
}

wind_data_params = {
    "dataset_id": "cmems_obs-wind_glo_phy_nrt_l4_0.125deg_PT1H",
    "variables": ["eastward_wind", "northward_wind"],
    "minimum_longitude": 71.67651854792956,
    "maximum_longitude": 83.99011977588334,
    "minimum_latitude": 5.554121417215164,
    "maximum_latitude": 17.074981057486614,
    "start_datetime":a_wind,
    "end_datetime": a_wind,
    "username": "asas",
    "password": "dssd"
}

current_data_2_params = {
    "dataset_id": "cmems_mod_glo_phy-thetao_anfc_0.083deg_P1D-m",
    "variables": ["thetao"],
    "minimum_longitude": 71.67651854792956,
    "maximum_longitude": 83.99011977588334,
    "minimum_latitude": 5.554121417215164,
    "maximum_latitude": 17.074981057486614,
    "start_datetime": a,
    "end_datetime": a,
    "minimum_depth": 0.49402499198913574,
    "maximum_depth": 0.49402499198913574,
    "username": "asas",
    "password": "dssd"
}

# Run each query separately:
print("Fetching current_data:")
current_data_df = fetch_and_display_data(current_data_params)
push_data_to_bigquery(current_data_df, dataset_id, "current_data")

print("\nFetching biogeochemical_data:")
biogeochemical_data_df = fetch_and_display_data(biogeochemical_data_params)
push_data_to_bigquery(biogeochemical_data_df, dataset_id, "biogeochemical_data")

print("\nFetching wind_data:")
wind_data_df = fetch_and_display_data(wind_data_params)
push_data_to_bigquery(wind_data_df, dataset_id, "wind_data")

print("\nFetching current_data_2:")
current_data_2_df = fetch_and_display_data(current_data_2_params)
push_data_to_bigquery(current_data_2_df, dataset_id, "current_data_2")

Fetching current_data:


INFO - 2025-04-10T20:39:56Z - Selected dataset version: "202406"
INFO:copernicusmarine:Selected dataset version: "202406"
INFO - 2025-04-10T20:39:56Z - Selected dataset part: "default"
INFO:copernicusmarine:Selected dataset part: "default"


Data fetched successfully:
          depth   latitude  longitude       time        uo        vo
0      0.494025   5.583336  71.750000 2025-04-11  0.085712 -0.206032
1      0.494025   5.583336  71.833344 2025-04-11  0.052188 -0.292032
2      0.494025   5.583336  71.916672 2025-04-11  0.016429 -0.368978
3      0.494025   5.583336  72.000000 2025-04-11 -0.012949 -0.423768
4      0.494025   5.583336  72.083344 2025-04-11 -0.038140 -0.471999
...         ...        ...        ...        ...       ...       ...
20281  0.494025  17.000000  83.583344 2025-04-11  0.051357  0.041299
20282  0.494025  17.000000  83.666687 2025-04-11 -0.009885 -0.012147
20283  0.494025  17.000000  83.750000 2025-04-11 -0.060266 -0.061246
20284  0.494025  17.000000  83.833344 2025-04-11 -0.097096 -0.084607
20285  0.494025  17.000000  83.916687 2025-04-11 -0.129645 -0.099479

[13326 rows x 6 columns]
Pushed data to ocean-data-e68c2.ocean_data.current_data

Fetching biogeochemical_data:


INFO - 2025-04-10T20:40:19Z - Selected dataset version: "202311"
INFO:copernicusmarine:Selected dataset version: "202311"
INFO - 2025-04-10T20:40:19Z - Selected dataset part: "default"
INFO:copernicusmarine:Selected dataset part: "default"


Data fetched successfully:
         depth  latitude  longitude       time      nppv          o2
0     0.494025      5.75      71.75 2025-04-11  5.126607  201.555084
1     0.494025      5.75      72.00 2025-04-11  4.168730  200.748550
2     0.494025      5.75      72.25 2025-04-11  4.513958  200.276367
3     0.494025      5.75      72.50 2025-04-11  5.518563  200.506500
4     0.494025      5.75      72.75 2025-04-11  5.744860  200.818024
...        ...       ...        ...        ...       ...         ...
2249  0.494025     17.00      82.75 2025-04-11  9.775278  204.352951
2250  0.494025     17.00      83.00 2025-04-11  8.713862  204.348145
2251  0.494025     17.00      83.25 2025-04-11  8.028949  204.287323
2252  0.494025     17.00      83.50 2025-04-11  7.671774  203.875412
2253  0.494025     17.00      83.75 2025-04-11  7.927057  204.096436

[1457 rows x 6 columns]
Pushed data to ocean-data-e68c2.ocean_data.biogeochemical_data

Fetching wind_data:


INFO - 2025-04-10T20:40:42Z - Selected dataset version: "202207"
INFO:copernicusmarine:Selected dataset version: "202207"
INFO - 2025-04-10T20:40:42Z - Selected dataset part: "default"
INFO:copernicusmarine:Selected dataset part: "default"


Data fetched successfully:
                    time  latitude  longitude  eastward_wind  northward_wind
0    2025-04-08 21:00:00    5.5625    71.6875           5.07            0.73
1    2025-04-08 21:00:00    5.5625    71.8125           5.13            0.65
2    2025-04-08 21:00:00    5.5625    71.9375           5.10            0.71
3    2025-04-08 21:00:00    5.5625    72.0625           5.16            0.75
4    2025-04-08 21:00:00    5.5625    72.1875           5.22            0.72
...                  ...       ...        ...            ...             ...
9202 2025-04-08 21:00:00   17.0625    83.4375          -5.19           -1.51
9203 2025-04-08 21:00:00   17.0625    83.5625          -5.33           -1.21
9204 2025-04-08 21:00:00   17.0625    83.6875          -5.46           -0.95
9205 2025-04-08 21:00:00   17.0625    83.8125          -5.66           -0.44
9206 2025-04-08 21:00:00   17.0625    83.9375          -5.87           -0.10

[9207 rows x 5 columns]
Pushed data to ocean-dat

INFO - 2025-04-10T20:41:07Z - Selected dataset version: "202406"
INFO:copernicusmarine:Selected dataset version: "202406"
INFO - 2025-04-10T20:41:07Z - Selected dataset part: "default"
INFO:copernicusmarine:Selected dataset part: "default"


Data fetched successfully:
          depth   latitude  longitude       time     thetao
0      0.494025   5.583336  71.750000 2025-04-11  30.080336
1      0.494025   5.583336  71.833344 2025-04-11  30.014435
2      0.494025   5.583336  71.916672 2025-04-11  30.051800
3      0.494025   5.583336  72.000000 2025-04-11  30.231216
4      0.494025   5.583336  72.083344 2025-04-11  30.378590
...         ...        ...        ...        ...        ...
20281  0.494025  17.000000  83.583344 2025-04-11  30.065073
20282  0.494025  17.000000  83.666687 2025-04-11  30.023327
20283  0.494025  17.000000  83.750000 2025-04-11  29.950909
20284  0.494025  17.000000  83.833344 2025-04-11  29.894344
20285  0.494025  17.000000  83.916687 2025-04-11  29.865007

[13326 rows x 5 columns]
Pushed data to ocean-data-e68c2.ocean_data.current_data_2


In [20]:
#wind data
from google.cloud import bigquery
import copernicusmarine
import pandas as pd
from datetime import datetime

# Get the current timestamp in the required format
current_time = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")

# Initialize BigQuery client
client = bigquery.Client()
dataset_id = "ocean-data-e68c2.ocean_data"

# Function to fetch and display wind data
def fetch_wind_data():
    wind_data_params = {
        "dataset_id": "cmems_obs-wind_glo_phy_nrt_l3-hy2b-hscat-asc-0.25deg_P1D-i",
        "variables": ["eastward_wind", "northward_wind"],
        "minimum_longitude": 71.67651854792956,
        "maximum_longitude": 83.99011977588334,
        "minimum_latitude": 5.554121417215164,
        "maximum_latitude": 17.074981057486614,
        "start_datetime": current_time,
        "end_datetime": current_time,
        "username": "asas",
        "password": "dssd"
    }

    try:
        df = copernicusmarine.read_dataframe(**wind_data_params).reset_index().dropna()
        if not df.empty:
            print("Wind Data Fetched Successfully:")
            print(df)
            return df
        else:
            print("No wind data found for the specified parameters.")
            return None
    except Exception as e:
        print(f"Error fetching wind data: {e}")
        return None

# Fetch wind data
wind_data_df = fetch_wind_data()


INFO - 2025-04-10T20:41:29Z - Selected dataset version: "202311"
INFO:copernicusmarine:Selected dataset version: "202311"
INFO - 2025-04-10T20:41:29Z - Selected dataset part: "default"
INFO:copernicusmarine:Selected dataset part: "default"


Error fetching wind data: Some of your subset selection [2025-04-10 20:41:26+00:00, 2025-04-10 20:41:26+00:00] for the time dimension exceed the dataset coordinates [2020-11-11 00:00:00+00:00, 2025-04-09 00:00:00+00:00]


In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from geopy.distance import geodesic
from itertools import combinations
import pandas_gbq
from google.cloud import bigquery
# Step 1: Define base URL and target URL
base_url = "https://incois.gov.in/MarineFisheries/"
target_url = "https://incois.gov.in/MarineFisheries/TextDataHome?mfid=1&request_locale=en#"

# Step 2: Set headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
}

# Step 3: Fetch the main page
session = requests.Session()
response = session.get(target_url, headers=headers)

# Step 4: Parse the HTML content using BeautifulSoup
if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")

    # Step 5: Find the link to "SOUTH TAMILNADU"
    south_tn_link = None
    for link in soup.find_all("a", href=True):
        if "NORTH TAMILNADU" in link.text:
            south_tn_link = base_url + link["href"]
            break

    # Step 6: If the link is found, fetch the data
    if south_tn_link:
        south_tn_response = session.get(south_tn_link, headers=headers)

        if south_tn_response.status_code == 200:
            pass
            #print("Data from SOUTH TAMILNADU page:\n")
            #print(south_tn_response.text)  # Prints extracted page content
        else:
            print(f"Failed to load SOUTH TAMILNADU page. Status Code: {south_tn_response.status_code}")
    else:
        print("Could not find the SOUTH TAMILNADU link on the page.")
else:
    print(f"Failed to fetch the main page. Status Code: {response.status_code}")

html_content = south_tn_response.text

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

# Find all table rows
rows = soup.find_all("tr")

# Initialize lists for storing latitude and longitude
latitudes = []
longitudes = []

# Iterate over rows and extract latitude & longitude
for row in rows:
    cells = row.find_all("td")
    if len(cells) >= 7:  # Ensure there are enough columns
        latitudes.append(cells[-2].text.strip())  # Second last column for latitude
        longitudes.append(cells[-1].text.strip())  # Last column for longitude

# Convert to a Pandas DataFrame
df = pd.DataFrame({"Latitude": latitudes, "Longitude": longitudes})

def dms_to_dd(dms_str):
    parts = dms_str.split()
    degrees = float(parts[0])
    minutes = float(parts[1])
    seconds = float(parts[2])
    direction = parts[3]  # 'N', 'S', 'E', 'W'

    decimal = degrees + (minutes / 60) + (seconds / 3600)

    # Convert to negative for South and West directions
    if direction in ['S', 'W']:
        decimal = -decimal

    return decimal

# Convert Latitude and Longitude to Decimal Degrees
df['Latitude_DD'] = df['Latitude'].apply(dms_to_dd)
df['Longitude_DD'] = df['Longitude'].apply(dms_to_dd)



lat_col = "Latitude_DD"  # Replace with actual column name if different
lon_col = "Longitude_DD"  # Replace with actual column name if different

# Create a list to store WKT lines
wkt_lines = []
line_counter = 1  # Naming the lines sequentially

# Iterate through all unique pairs of coordinates
for (index1, row1), (index2, row2) in combinations(df.iterrows(), 2):
    coord1 = (row1[lat_col], row1[lon_col])
    coord2 = (row2[lat_col], row2[lon_col])

    # Calculate distance between two points
    distance = geodesic(coord1, coord2).kilometers

    # Connect points if they are within 3.5 km
    if distance <= 5:
        wkt_format = f"LINESTRING ({row1[lon_col]} {row1[lat_col]}, {row2[lon_col]} {row2[lat_col]})"
        wkt_lines.append([wkt_format, f"Line {line_counter}", ""])
        line_counter += 1

# Create a DataFrame with WKT format
wkt_df1 = pd.DataFrame(wkt_lines, columns=["WKT", "name", "description"])



base_url = "https://incois.gov.in/MarineFisheries/"
target_url = "https://incois.gov.in/MarineFisheries/TextDataHome?mfid=1&request_locale=en#"

# Step 2: Set headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
}

# Step 3: Fetch the main page
session = requests.Session()
response = session.get(target_url, headers=headers)

# Step 4: Parse the HTML content using BeautifulSoup
if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")

    # Step 5: Find the link to "SOUTH TAMILNADU"
    south_tn_link = None
    for link in soup.find_all("a", href=True):
        if "SOUTH TAMILNADU" in link.text:
            south_tn_link = base_url + link["href"]
            break

    # Step 6: If the link is found, fetch the data
    if south_tn_link:
        south_tn_response = session.get(south_tn_link, headers=headers)

        if south_tn_response.status_code == 200:
            pass
            #print("Data from SOUTH TAMILNADU page:\n")
            #print(south_tn_response.text)  # Prints extracted page content
        else:
            print(f"Failed to load SOUTH TAMILNADU page. Status Code: {south_tn_response.status_code}")
    else:
        print("Could not find the SOUTH TAMILNADU link on the page.")
else:
    print(f"Failed to fetch the main page. Status Code: {response.status_code}")

html_content = south_tn_response.text

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

# Find all table rows
rows = soup.find_all("tr")

# Initialize lists for storing latitude and longitude
latitudes = []
longitudes = []

# Iterate over rows and extract latitude & longitude
for row in rows:
    cells = row.find_all("td")
    if len(cells) >= 7:  # Ensure there are enough columns
        latitudes.append(cells[-2].text.strip())  # Second last column for latitude
        longitudes.append(cells[-1].text.strip())  # Last column for longitude

# Convert to a Pandas DataFrame
df = pd.DataFrame({"Latitude": latitudes, "Longitude": longitudes})

def dms_to_dd(dms_str):
    parts = dms_str.split()
    degrees = float(parts[0])
    minutes = float(parts[1])
    seconds = float(parts[2])
    direction = parts[3]  # 'N', 'S', 'E', 'W'

    decimal = degrees + (minutes / 60) + (seconds / 3600)

    # Convert to negative for South and West directions
    if direction in ['S', 'W']:
        decimal = -decimal

    return decimal

# Convert Latitude and Longitude to Decimal Degrees
df['Latitude_DD'] = df['Latitude'].apply(dms_to_dd)
df['Longitude_DD'] = df['Longitude'].apply(dms_to_dd)



lat_col = "Latitude_DD"  # Replace with actual column name if different
lon_col = "Longitude_DD"  # Replace with actual column name if different

# Create a list to store WKT lines
wkt_lines = []
line_counter = 1  # Naming the lines sequentially

# Iterate through all unique pairs of coordinates
for (index1, row1), (index2, row2) in combinations(df.iterrows(), 2):
    coord1 = (row1[lat_col], row1[lon_col])
    coord2 = (row2[lat_col], row2[lon_col])

    # Calculate distance between two points
    distance = geodesic(coord1, coord2).kilometers

    # Connect points if they are within 3.5 km
    if distance <= 5:
        wkt_format = f"LINESTRING ({row1[lon_col]} {row1[lat_col]}, {row2[lon_col]} {row2[lat_col]})"
        wkt_lines.append([wkt_format, f"Line {line_counter}", ""])
        line_counter += 1

# Create a DataFrame with WKT format
wkt_df2 = pd.DataFrame(wkt_lines, columns=["WKT", "name", "description"])
merged_df = pd.concat([wkt_df1, wkt_df2], ignore_index=True)
merged_df['name'] = ['Line ' + str(i+1) for i in range(len(merged_df))]
output_file = "FINAL_LINES.csv"
client = bigquery.Client()
dataset_id = "ocean-data-e68c2.ocean_data"
table_name = "pfz"

try:
    pandas_gbq.to_gbq(merged_df, f"{dataset_id}.{table_name}", project_id=client.project, if_exists='replace')
    print(f"DataFrame pushed to {dataset_id}.{table_name} successfully.")
except Exception as e:
    print(f"Error pushing DataFrame to BigQuery: {e}")

DataFrame pushed to ocean-data-e68c2.ocean_data.pfz successfully.


In [ ]:
import requests
from io import BytesIO
from pdf2image import convert_from_bytes
from PIL import Image
import base64
from google.cloud import bigquery
import google.generativeai as genai
import os

# ---------- CONFIG ----------
GEMINI_API_KEY = 'YOUR_GEMINI_API_KEY'  # Replace with your actual API key
# PDF URL
PDF_URL = "https://mausam.imd.gov.in/chennai/mcdata/fishermen.pdf"
BIGQUERY_TABLE = "ocean-data-e68c2.ocean_data.link"

# ---------- AUTH ----------
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY
genai.configure(api_key=GEMINI_API_KEY)

# Use gemini-pro-vision for image input
model = genai.GenerativeModel("gemini-2.0-flash-thinking-exp-01-21")

# BigQuery client
bq_client = bigquery.Client()

# ---------- DOWNLOAD PDF ----------
print("📥 Downloading PDF...")
response = requests.get(PDF_URL)
pdf_data = BytesIO(response.content)

# ---------- CONVERT TO IMAGES ----------
print("🖼️ Converting PDF pages to images...")
images = convert_from_bytes(pdf_data.read())

# ---------- PROCESS EACH PAGE ----------
rows_to_insert = []

for i, img in enumerate(images, start=1):
    print(f"🔍 Processing page {i}...")

    # Convert image to bytes
    buffered = BytesIO()
    img.save(buffered, format="PNG")
    image_bytes = buffered.getvalue()

    # Encode image to base64 for BigQuery storage
    img_base64 = base64.b64encode(image_bytes).decode("utf-8")

    # Gemini Summary - English
    try:
        english_summary = model.generate_content([
            "Summarize this weather bulletin page in English clearly:",
            {"mime_type": "image/png", "data": image_bytes}
        ]).text.strip()
    except Exception as e:
        english_summary = f"Error generating English summary: {str(e)}"

    # Gemini Summary - Tamil
    try:
        tamil_summary = model.generate_content([
            "இது ஒரு வானிலை அறிவிப்பு பக்கம். தயவுசெய்து இதை தமிழில் சுருக்கமாக விவரிக்கவும்:",
            {"mime_type": "image/png", "data": image_bytes}
        ]).text.strip()
    except Exception as e:
        tamil_summary = f"தமிழ் சுருக்கம் பெற முடியவில்லை: {str(e)}"

    # Prepare row for BigQuery
    row = {
        "id": i,
        "english": english_summary,
        "tamil": tamil_summary,
        "image": img_base64
    }
    rows_to_insert.append(row)

# ---------- PUSH TO BIGQUERY ----------
print("🚀 Uploading to BigQuery...")
errors = bq_client.insert_rows_json(BIGQUERY_TABLE, rows_to_insert)

if not errors:
    print("✅ All data inserted successfully into BigQuery!")
else:
    print("❌ Errors occurred while inserting into BigQuery:")
    print(errors)


📥 Downloading PDF...
🖼️ Converting PDF pages to images...
🔍 Processing page 1...
🔍 Processing page 2...


ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash-thinking-exp-01-21:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 3146.20ms


🔍 Processing page 3...


ERROR:tornado.access:503 POST /v1beta/models/gemini-2.0-flash-thinking-exp-01-21:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2892.40ms


🔍 Processing page 4...
🔍 Processing page 5...
🚀 Uploading to BigQuery...
✅ All data inserted successfully into BigQuery!
